In [1]:
import pandas as pd
import time

In [2]:
start=time.perf_counter()

In [3]:
from DatabaseCreator import DatabaseCreator
import CJDH_local_settings

#Run Database Creator
if __name__ == "__main__":
    db_creator = DatabaseCreator(db_settings=CJDH_local_settings.local_settings['FPL_Points_Predictor'])
    fpl_engine = db_creator.get_engine_for("fpl_data_analysis")

In [4]:
#add this info to a settings dict?
stats_current_gameweek = 9
picks_current_gameweek = 11
current_season = 20252026

In [5]:
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        in_my_team,
        position,
        team_name,
        ROUND(AVG(opp_team_elo),1) AS avg_opp_team_elo,
        ROUND(SUM(xpoints),2) AS total_xpoints,
        ROUND(AVG(mins_pred),1) AS avg_mins_pred,
        ROUND(AVG(xgp90_pred),3) AS avg_xgp90_pred,
        ROUND(AVG(xap90_pred),3) AS avg_xap90_pred,
        ROUND(AVG(xgcp90_pred),3) AS avg_xgcp90_pred
    FROM predictions
    WHERE (in_any_team = 0 OR in_my_team = 1)
      AND event BETWEEN {picks_current_gameweek} AND {picks_current_gameweek+5}
      AND position = 'MID'
    GROUP BY player_name_id, position, team_name, in_my_team
    ORDER BY total_xpoints DESC
""")
view.head(20)


,player_name_id,in_my_team,position,team_name,avg_opp_team_elo,total_xpoints,avg_mins_pred,avg_xgp90_pred,avg_xap90_pred,avg_xgcp90_pred
0,Bruno Borges Fernandes,1,MID,Man Utd,1023.5,34.17,78.6,0.365,0.257,1.294
1,Bruno GuimarÃ£es Rodriguez Moura,0,MID,Newcastle,1049.1,29.28,80.3,0.238,0.188,1.283
2,Dominik Szoboszlai,1,MID,Liverpool,1045.8,29.17,80.0,0.200,0.215,1.299
3,Amad Diallo,1,MID,Man Utd,1023.5,28.20,60.4,0.310,0.142,1.074
4,Yankuba Minteh,1,MID,Brighton,1084.4,27.85,81.8,0.202,0.171,1.460
5,Dango Ouattara,1,MID,Bournemouth,1066.6,27.49,71.0,0.286,0.096,1.384
6,Marcus Tavernier,0,MID,Bournemouth,1066.6,27.34,80.6,0.173,0.177,1.286
7,MartÃ­n Zubimendi IbÃ¡Ã±ez,0,MID,Arsenal,1053.8,26.99,77.2,0.130,0.174,0.838
8,Carlos Henrique Casimiro,0,MID,Man Utd,1023.5,26.97,65.4,0.255,0.116,1.155
9,Tijjani Reijnders,0,MID,Man City,1063.1,26.62,65.6,0.209,0.132,1.012


In [6]:
# #Gt member player codes from the latest GW
# query = """
#         WITH
#         currentgw AS (
#             SELECT MAX(gw) as max_gw
#             FROM playergw)
#         SELECT *
#         FROM playergw
#         WHERE gw = currentgw
#         ;
#         """
# with fpl_engine.connect() as conn:
#     viewdf = pd.read_sql_query(query, conn)
# print(viewdf.shape)

In [7]:
# #Query gets only player details from the latest GW
# query = """
#         ALTER TABLE player_details
#             RENAME COLUMN total_points TO total_points_gw;

#         WITH playergw AS (
#             SELECT *,
#             ROW_NUMBER() OVER(
#                     PARTITION BY
#                         season,
#                         code,
#                         event,
#                         fixture_id --Only one record per GW
#                     ORDER BY 
#                         now_datetime_utc DESC) as rn --Only select the latest date
#         FROM playergw
#         ),
#         currentgw AS (
#             SELECT MAX(current_gw) as max_gw
#             FROM playergw
#         )
        
#         SELECT *
#         FROM player_details
#         LEFT JOIN playergw
#             ON player_details.element = playergw.id
#             AND player_details.round = playergw.current_gw
#         CROSS JOIN currentgw
#         WHERE player_details.round = currentgw.max_gw
#             AND playergw.rn = 1
#         ;
#         """
# with engine.connect() as conn:
#     playerdf = pd.read_sql_query(query, conn)
# print(playerdf.shape)

# pd.pivot_table(playerdf, index=['id','web_name','team_name','round'],values='total_points_gw',aggfunc=np.sum).sort_values(by='total_points_gw', ascending=False)

In [8]:
# max_gw = max(picksdf['gw'])
# me = 'Christopher Harris'
# myid = set(picksdf[picksdf['member']==me]['memberid'])
# mypicksdf = picksdf[(picksdf['member']==me)&(picksdf['gw']==max_gw)]
# mypicks = set(mypicksdf['element'])
# mypicks

# #league_standings
# l = pd.DataFrame(league_data['standings']['results'])
# mytotal = l[l['entry'].isin(myid)]['total'].values
# #get ids for members above me or less than 100 points below me
# ids_close_to_me = set(l[l['total']<=int(mytotal)+100]['entry'][~l['entry'].isin(myid)])

In [9]:
# #Get ep_next * by ownership rate in mini-league
# latestpicks = picksdf[(picksdf['gw']==max(picksdf['gw']))&(picksdf['memberid'].isin(ids_close_to_me))]
# max_owns = len(set(latestpicks['memberid']))

# ownership = pd.pivot_table(latestpicks, index=['element'],values='multiplier',aggfunc=np.sum).reset_index()
# ownership['own_rate'] = ownership['multiplier']/max_owns

# ownership.sort_values(by='own_rate', ascending=False)

# playerdf = pd.merge(playerdf, ownership, how='outer',on='element')
# filtered = playerdf['element'].isin(mypicks)
# gainpicks = playerdf[filtered].merge(mypicksdf,how='left',on='element',suffixes=('','_myteam'))

# gainpicks['expected_gain'] = gainpicks['ep_next'].astype(float)*(1-gainpicks['own_rate']) #change 1 for "gainpicks['multiplier_myteam']" if team multiplier needed
# gainpicks[['web_name','element','element_type','multiplier','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False) #'multiplier_myteam'

In [10]:
# print("Expected Total Gain on Division Average")
# gainpicks['expected_gain'].sum()

In [11]:
# #players not in my picks
# losspicks = playerdf[~filtered].copy().dropna(subset='multiplier')
# losspicks['expected_loss'] = losspicks['ep_next'].astype(float)*(0-losspicks['own_rate'])
# print("Expected Total Loss on Division Average: ")
# print(losspicks['expected_loss'].sum())
# losspicks[['web_name','element','multiplier','own_rate','ep_next','expected_loss']].sort_values(by='expected_loss',ascending=True).head()

In [12]:
# print("Net Gain")
# print(round(gainpicks['expected_gain'].sum() + losspicks['expected_loss'].sum(),2))

In [13]:
# teamgains = gainpicks[['web_name','status','element',
#                        'element_type','now_cost','multiplier',
#                        'multiplier_myteam','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False)
# teamgains['expected_loss_on_transfer'] = teamgains['ep_next'].astype(float)*(teamgains['own_rate'])

# allgains = playerdf.copy()
# allgains['expected_gain'] = allgains['ep_next'].astype(float)*(1-allgains['own_rate'])

# def top_suggested_transfer(row, df, current_team):
#     #get element, element_type & gain
#     myteam_type = row['element_type']
#     # myteam_gain = row['expected_gain']
#     myteam_loss = row['expected_loss_on_transfer']
#     myteam_cost = row['now_cost']
#     myteam_element = row['element']
#     #get playerdf element, element_type & gain
#     filtered = (df['element_type']==myteam_type) & (df['expected_gain']>myteam_loss) & (~df['element'].isin(current_team)) & (df['now_cost']<=myteam_cost)
#     best_gain = df[filtered].sort_values(by='expected_gain', ascending=False).head(1)
#     best_gain['own_rate'] = round(best_gain['own_rate'],3)
#     best_gain['expected_gain'] = round(best_gain['expected_gain'],2)
#     best_gain['diff'] = round(df['expected_gain']-row['expected_loss_on_transfer'],2)
#     return best_gain[['web_name','status','own_rate','ep_next','expected_gain','diff']].values[0]

# current_team = set(gainpicks['element'])
# teamgains['affordable_suggested_transfer'] = teamgains.apply(lambda x: top_suggested_transfer(x, allgains, current_team),axis=1)
# # Extracting the second element from the arrays in the "affordable_suggested_transfer" column
# teamgains['suggested_transfer_gain'] = teamgains['affordable_suggested_transfer'].apply(lambda x: x[-1])

# # Sorting the DataFrame based on the second element in the arrays
# teamgains['gain_diff'] = teamgains['suggested_transfer_gain'] - teamgains['expected_gain'] - teamgains['expected_loss_on_transfer']
# teamgains = teamgains.sort_values(by='gain_diff', ascending=False)

# teamgains.head(20)

In [14]:
# print("Sorted by Expected Gain")
# allgains[['web_name','status','element','element_type','now_cost','ep_next','own_rate','expected_gain']].sort_values(by='expected_gain',ascending=False).dropna().head(20)

In [ ]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#10/11/25: Finished in 0.01 minute(s)

Finished in 0.01 minute(s)
